# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Pancreatic Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
# NORMALIZE_GENE = False
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [14]:
trait_subdir = "TCGA_Pancreatic_Cancer_(PAAD)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.PAAD.sampleMap_PAAD_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.PAAD.sampleMap_HiSeqV2_PANCAN.gz')

In [15]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [16]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [17]:
clinical_data_cols[:10]

['CDE_ID_3226963',
 '_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'adenocarcinoma_invasion',
 'age_at_initial_pathologic_diagnosis']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [18]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['CDE_ID_3226963', '_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'adenocarcinoma_invasion', 'age_at_initial_pathologic_diagnosis', 'alcohol_history_documented', 'alcoholic_exposure_category', 'amount_of_alcohol_consumption_per_day', 'anatomic_neoplasm_subdivision', 'anatomic_neoplasm_subdivision_other', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days

In [19]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [20]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [65, 48, 75, 71, 70],
 'days_to_birth': [-23962, -17794, -27600, -26028, -25920]}

In [21]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [22]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [23]:
gender_col = 'gender'

In [24]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [25]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

15 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('SNORD104', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
9 input query terms found dup hits:	[('SUGT1P1', 2), ('SNORD127', 2), ('SNORA62', 4), ('IFITM4P', 8), ('HLA-DRB6', 2), ('FUNDC2P2', 2), 
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
20 input query terms found dup hits:	[('PIP5K1P1', 2), ('SNORA40', 2), ('SNORD116-24', 2), ('SNORD116-25', 2), ('SNORD116-23', 2), ('SNOR
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
18 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('SNORD116-22', 2), (
158 input query terms found no hit:	['

In [26]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Pancreatic Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-2J-AAB1-01,1,65,1,-0.072992,3.176274,0.042165,0.133528,-0.736678,-0.19891,0.071506,...,-2.090786,-0.086682,-0.748878,-0.604817,0.599267,-1.28139,-0.223977,0.385573,0.775305,2.372667
TCGA-2J-AAB4-01,1,48,1,0.654608,0.380374,-0.531035,-0.046472,-0.185578,-0.50201,0.326006,...,-1.571286,-0.086682,0.675222,0.672283,0.434167,-1.28139,-0.609577,0.170673,0.535405,1.988767
TCGA-2J-AAB6-01,1,75,1,0.377208,1.086674,-0.531035,0.463928,0.479622,-0.47171,-0.190094,...,-2.090786,-0.086682,-0.748878,-1.587117,0.744867,-1.28139,-0.659977,0.160973,0.588205,-0.817033
TCGA-2J-AAB8-01,1,71,1,-0.168392,-0.833226,-0.531035,-0.233272,-0.373978,-0.32791,-0.438694,...,-2.090786,-0.086682,-0.748878,-0.061017,0.140167,0.65221,0.114023,0.864373,0.322105,1.441267
TCGA-2J-AAB9-01,1,70,0,0.563508,0.298374,0.519865,0.089028,-0.273578,-0.32561,0.539906,...,-1.039886,-0.086682,-0.748878,1.963583,0.751967,0.37101,-0.000777,1.271773,-0.053795,3.029567


In [27]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 183 samples.


In [28]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Pancreatic Cancer', the least common label is '0' with 4 occurrences. This represents 2.19% of the dataset.
The distribution of the feature 'Pancreatic Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 57.0
  50% (Median): 65.0
  75%: 73.0
Min: 35
Max: 88
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 82 occurrences. This represents 44.81% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [29]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [30]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [31]:
dataset = 'GEO'
trait_subdir = "Pancreatic-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE236951', 'GSE212211', 'GSE223409']

Repeat the below steps for all the accession numbers

In [32]:
cohort = accession_num = "GSE236951"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Pancreatic-Cancer/GSE236951/GSE236951_family.soft.gz',
 '../../DATA/GEO/Pancreatic-Cancer/GSE236951/GSE236951_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [33]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Gene based immune profiling of vermiform appendix GALT of adult patients with pancreatic cancer or colon diseases."
!Series_summary	"Differential immune gene expression analysis in appendix GALT between patients with pancreatic cancer versus benign or malignant colon disease using the nanostring ncounter platform."
!Series_overall_design	"In this study the GALT of appendix was analyzed by RNA expression analysis of ~700 immune related genes using the Pan Cancer Immune Panel from nanostring. Non inflamed normal looking FFPE appendix tissue from sixteen pancreatic cancer patients, nine colon cancer patients, and nine benign colon disease patients was used to compare GALT immune profiles among the three groups."


In [34]:
clinical_data.head()

,!Sample_geo_accession,GSM7587683,GSM7587684,GSM7587685,GSM7587686,GSM7587687,GSM7587688,GSM7587689,GSM7587690,GSM7587691,...,GSM7587707,GSM7587708,GSM7587709,GSM7587710,GSM7587711,GSM7587712,GSM7587713,GSM7587714,GSM7587715,GSM7587716
0,!Sample_characteristics_ch1,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,disease: Pancreatic ductal adenocarcinoma,...,disease: Colon adenocarcinoma,disease: Colon adenocarcinoma,disease: Benign colon disease,disease: Benign colon disease,disease: Benign colon disease,disease: Benign colon disease,disease: Benign colon disease,disease: Benign colon disease,disease: Benign colon disease,disease: Benign colon disease
1,!Sample_characteristics_ch1,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: None,...,neoadjuvant therapy: None,neoadjuvant therapy: None,neoadjuvant therapy: n.a.,neoadjuvant therapy: n.a.,neoadjuvant therapy: n.a.,neoadjuvant therapy: n.a.,neoadjuvant therapy: n.a.,neoadjuvant therapy: n.a.,neoadjuvant therapy: n.a.,neoadjuvant therapy: n.a.
2,!Sample_characteristics_ch1,Sex: Male,Sex: Male,Sex: Male,Sex: Male,Sex: Female,Sex: Male,Sex: Male,Sex: Female,Sex: Female,...,Sex: Female,Sex: Female,Sex: Male,Sex: Male,Sex: Female,Sex: Female,Sex: Female,Sex: Male,Sex: Male,Sex: Female
3,!Sample_characteristics_ch1,age: 83 years,age: 64 years,age: 59 years,age: 64 years,age: 72 years,age: 72 years,age: 89 years,age: 59 years,age: 64 years,...,age: 53 years,age: 95 years,age: 53 years,age: 55 years,age: 43 years,age: 71 years,age: 48 years,age: 43 years,age: 55 years,age: 63 years


In [35]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['disease: Pancreatic ductal adenocarcinoma',
  'disease: Colon adenocarcinoma',
  'disease: Benign colon disease'],
 1: ['neoadjuvant therapy: None', 'neoadjuvant therapy: n.a.'],
 2: ['Sex: Male', 'Sex: Female'],
 3: ['age: 83 years',
  'age: 64 years',
  'age: 59 years',
  'age: 72 years',
  'age: 89 years',
  'age: 82 years',
  'age: 75 years',
  'age: 61 years',
  'age: 68 years',
  'age: 49 years',
  'age: 71 years',
  'age: 58 years',
  'age: 76 years',
  'age: 67 years',
  'age: 52 years',
  'age: 57 years',
  'age: 53 years',
  'age: 95 years',
  'age: 55 years',
  'age: 43 years',
  'age: 48 years',
  'age: 63 years']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [36]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Pancreatic Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Pancreatic Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [37]:
age_row = gender_row = None
convert_age = convert_gender = None

In [38]:
is_gene_availabe = True
trait_row = 0
age_row = 3
gender_row = 2

trait_type = 'binary'

In [39]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [40]:
import re
def convert_trait(value):
    if value == 'disease: Pancreatic ductal adenocarcinoma':
        return 1
    elif value == 'disease: Benign colon disease':
        return 0
    else:
        return None

def convert_age(trait_value):
    age_match = re.search(r'\d+ years', trait_value)
    if age_match:
        return int(age_match.group(0).split()[0])
    else:
        return None


def convert_gender(value):
    """
    Convert the gender value to a binary format.
    'M' for male is converted to 1, and 'F' for female is converted to 0.

    Args:
    value (str): The gender value in the format 'gender: X'.

    Returns:
    int: 1 for male, 0 for female, or None if the value is not in the expected format.
    """
    if value == 'Sex: Male':
        return 1
    elif value == 'Sex: Female':
        return 0
    else:
        return None

In [42]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM7587683,GSM7587684,GSM7587685,GSM7587686,GSM7587687,GSM7587688,GSM7587689,GSM7587690,GSM7587691,GSM7587692,...,GSM7587707,GSM7587708,GSM7587709,GSM7587710,GSM7587711,GSM7587712,GSM7587713,GSM7587714,GSM7587715,GSM7587716
Pancreatic Cancer,1,1,1,1,1,1,1,1,1,1,...,None,None,0,0,0,0,0,0,0,0
Age,83,64,59,64,72,72,89,59,64,82,...,53,95,53,55,43,71,48,43,55,63
Gender,1,1,1,1,0,1,1,0,0,0,...,0,0,1,1,0,0,0,1,1,0


### Genetic data preprocessing and final filtering

In [43]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM7587683,GSM7587684,GSM7587685,GSM7587686,GSM7587687,GSM7587688,GSM7587689,GSM7587690,GSM7587691,GSM7587692,...,GSM7587707,GSM7587708,GSM7587709,GSM7587710,GSM7587711,GSM7587712,GSM7587713,GSM7587714,GSM7587715,GSM7587716
ID,,,,,,,,,,,,,,,,,,,,,
A2M,13.071117,13.515604,13.288653,13.801596,13.154276,13.447439,13.099154,13.605908,12.490481,13.090440,...,13.332927,13.668196,13.398971,14.388423,13.251077,13.875481,14.004492,13.170939,12.307995,13.410477
ABCB1,9.946163,10.543342,9.608472,10.066367,12.152271,9.815143,10.630754,9.783274,9.565685,10.311819,...,9.685505,9.919031,10.883408,10.162817,10.044669,10.364624,10.376823,9.636499,9.932300,10.593647
ABL1,10.345667,10.548967,10.405202,10.278806,10.252386,10.146349,9.944674,10.272593,9.922946,9.962764,...,10.451650,10.962352,10.490571,11.255471,10.764135,10.720892,10.857599,10.140400,9.975869,10.681644
ADA,9.222631,8.920553,9.116346,9.124335,8.191202,9.436976,9.912189,8.895909,9.781248,9.207426,...,9.130332,8.216305,9.305459,8.478238,8.539324,9.273433,8.332429,9.188707,9.475842,8.423722
ADORA2A,8.779165,8.688000,8.618606,8.287657,7.185985,8.000877,9.574101,8.195099,8.388289,8.083000,...,9.090126,8.046844,8.186160,8.078707,8.268522,8.641164,8.477480,9.338207,9.553107,9.250391


In [44]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['A2M',
 'ABCB1',
 'ABL1',
 'ADA',
 'ADORA2A',
 'AICDA',
 'AIRE',
 'AKT3',
 'ALCAM',
 'AMBP',
 'AMICA1',
 'ANP32B',
 'ANXA1',
 'APOE',
 'APP',
 'ARG1',
 'ARG2',
 'ATF1',
 'ATF2',
 'ATG10']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [45]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['A2M', 'ABCB1', 'ABL1', 'ADA', 'ADORA2A', 'AICDA', 'AIRE', 'AKT3', 'ALCAM', 'AMBP', 'AMICA1', 'ANP32B', 'ANXA1', 'APOE', 'APP', 'ARG1', 'ARG2', 'ATF1', 'ATF2', 'ATG10']\n"

If not required, jump directly to the gene normalization step

In [47]:
# requires_gene_mapping = True
requires_gene_mapping = False

In [48]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [49]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_id_key = key_name1
gene_symb_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

NameError: name 'gene_annotation_summary' is not defined

In [23]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'gene_assignment'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [50]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

11 input query terms found no hit:	['AMICA1', 'CD3EAP', 'CD97', 'DDX58', 'IL8', 'KIR_Activating_Subgroup_1', 'KIR_Activating_Subgroup_2


In [51]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [52]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 24 samples.


In [53]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Pancreatic Cancer', the least common label is '0.0' with 8 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Pancreatic Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.0
  50% (Median): 63.5
  75%: 71.25
Min: 43.0
Max: 89.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 10 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [54]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [55]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [9]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
1,GSE236951,True,True,False,True,True,24.0,


In [10]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE236951'

In [11]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
1,GSE236951,True,True,False,True,True,24.0,


In [12]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Pancreatic Cancer,Age,Gender,A2M,ABCB1,ABL1,ADA,ADORA2A,AICDA,AIRE,...,ULBP2,USP9Y,VCAM1,VEGFA,VEGFC,XCL2,XCR1,YTHDF2,ZAP70,ZNF205
0,1.0,83.0,1.0,13.071117,9.946163,10.345667,9.222631,8.779165,8.143064,7.918190,...,7.969416,8.113007,10.768961,10.652329,7.781556,8.233149,7.571057,10.588228,10.051936,6.187007
1,1.0,64.0,1.0,13.515604,10.543342,10.548967,8.920553,8.688000,5.733167,8.585001,...,9.242047,8.604038,10.790676,10.306554,8.849404,9.474079,8.601334,10.401323,9.761300,6.682893
2,1.0,59.0,1.0,13.288653,9.608472,10.405202,9.116346,8.618606,7.618606,7.394289,...,5.434181,8.732306,11.873200,10.088492,8.271207,8.148677,6.965705,10.495958,10.757420,6.176384
3,1.0,64.0,1.0,13.801596,10.066367,10.278806,9.124335,8.287657,6.911790,5.911790,...,3.393942,8.296454,11.590041,9.756284,8.722413,8.728926,7.722413,10.608702,10.853373,6.360775
4,1.0,72.0,0.0,13.154276,12.152271,10.252386,8.191202,7.185985,5.536892,5.286914,...,5.721316,3.884815,9.185985,10.857180,8.031657,7.520404,5.777900,10.267285,8.282031,6.435012


In [13]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [14]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [15]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [16]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 50.00% ± 15.81%


In [17]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 75.00% ± 27.39%


In [19]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [20]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.2674
Found 23 genes with non-zero coefficients associated with the trait 'Pancreatic Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
    CCL5    -2.651168              2.651168
   TIGIT    -2.213783              2.213783
   C4BPA     2.043835              2.043835
    SPP1     1.620884              1.620884
 S100A12     1.313537              1.313537
    CD1B    -1.212615              1.212615
   IKBKE    -1.170780              1.170780
  CD40LG    -0.905088              0.905088
     ENG     0.866214              0.866214
  LILRA4    -0.839419              0.839419
   IL17B     0.744181              0.744181
   ICAM1    -0.674650              0.674650
    IRF7    -0.577229              0.577229
  IL18R1    -0.485028              0.485028
   IFIT2    -0.405250              0.405250
   CXCL2    -0.39212

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [21]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.2674
Found 0 significant genes associated with the trait 'Pancreatic Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
